In [1]:
# imports
import ocw.dataset_processor as dsp
import ocw.data_source.local as local
import datetime as dt
from netCDF4 import num2date, Dataset, MFDataset
import numpy as np
import matplotlib.pyplot as plt

# filepaths
data_home_dir = '/mnt/norgay/Datasets/'
obs_dir = data_home_dir+'Climate/Berkeley_Earth/Raw_NetCDF_1deg_x_1deg/TAVG/'
mod_hist_dir = data_home_dir+'Pinatubo/Total_Effect_Estimation/Data/raw/MPI-ESM-LR/tas/'

obs_files = [obs_dir+'Complete_TAVG_Daily_LatLong1_{}.nc'.format(y) for y in range(1880,2010,10)]
obs_files = obs_files[:2]    # for testing only (comment out for running all files)
mod_hist_file = mod_hist_dir+'tas_day_MPI-ESM-LR_historical_r1i1p1_18500101-18591231.nc'

In [2]:
# settings
mod_var = 'tas'
clim_var = 'climatology'
obs_var = 'temperature'

# get climatology
ds_clim = Dataset(obs_files[0],'r')
clim = ds_clim.variables['climatology'][:] + 273.15      # convert to Kelvin

# get reference grid
ds_grid = local.load_file(mod_hist_file,mod_var)
lats = ds_grid.lats
lons = ds_grid.lons
del ds_grid

In [3]:
# load observations 
ds_obs = local.load_dataset_from_multiple_netcdf_files(
    obs_var, time_name='date_number', file_list=obs_files)

NC file 1/2 /mnt/norgay/Datasets/Climate/Berkeley_Earth/Raw_NetCDF_1deg_x_1deg/TAVG/Complete_TAVG_Daily_LatLong1_1880.nc
19.1206
NC file 2/2 /mnt/norgay/Datasets/Climate/Berkeley_Earth/Raw_NetCDF_1deg_x_1deg/TAVG/Complete_TAVG_Daily_LatLong1_1890.nc
19.768
19.1206
19.768


In [4]:
days_of_year = np.array([],dtype=int)
for t in obs_files:
    days_of_year = np.append(days_of_year,Dataset(t).variables['day_of_year'][:].astype(int))
    
for x,d in enumerate(days_of_year):
    ds_obs.values[x,:,:] = ds_obs.values[x,:,:] + clim[d-1,:,:]

TODO:
    - regrid (and save?)
    - temporal subsets by month
    - load all of MPI reference model
    - create adjustment weights
    - load each MPI projection timeslice
    - bias correct